In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
data = pd.read_json("../data/dump.json")

In [3]:
data.shape

(7949, 19)

In [4]:
data.head()

,action,change_size,flags,geo_ip,hashtags,is_anon,is_bot,is_minor,is_new,is_unpatrolled,mentions,ns,page_title,parent_rev_id,rev_id,summary,timestamp,url,user
0,edit,377.0,None,NaN,[],False,False,False,False,False,[],Talk,Talk:Bajadasaurus,883357329.0,883356516.0,/* Images */,2019-02-14 22:43:57.810307,https://en.wikipedia.org/w/index.php?diff=8833...,FunkMonk
1,edit,-66.0,None,NaN,[],False,False,False,False,False,[],Main,The King of Fighters: Maximum Impact,883357327.0,883220913.0,None,2019-02-14 22:43:58.168588,https://en.wikipedia.org/w/index.php?diff=8833...,Benjaminkirsc
2,edit,-4635.0,B,NaN,[],False,True,False,False,False,[],User,User:AlexNewArtBot/WomensHistorySearchResult,883357322.0,883032645.0,"most recent results, added 113, removed 145, t...",2019-02-14 22:43:58.324169,https://en.wikipedia.org/w/index.php?diff=8833...,InceptionBot
3,edit,1.0,None,NaN,[],False,False,False,False,False,[],Template,Template:Samoa-party-stub,883357331.0,398617982.0,None,2019-02-14 22:43:58.613009,https://en.wikipedia.org/w/index.php?diff=8833...,HapHaxion
4,edit,37.0,None,NaN,[],False,False,False,False,False,[],Main,Persela Lamongan,883357330.0,883030131.0,None,2019-02-14 22:43:58.668958,https://en.wikipedia.org/w/index.php?diff=8833...,Afria Nuswantara


In [5]:
data.tail()

,action,change_size,flags,geo_ip,hashtags,is_anon,is_bot,is_minor,is_new,is_unpatrolled,mentions,ns,page_title,parent_rev_id,rev_id,summary,timestamp,url,user
7944,edit,-6.0,None,NaN,[],False,False,False,False,False,[],Template,Template:New Jersey State Senators,883364554.0,880713885.0,None,2019-02-14 23:43:53.182064,https://en.wikipedia.org/w/index.php?diff=8833...,Muboshgu
7945,edit,77.0,M,NaN,[],False,False,True,False,False,[],Main,Paulo Gazzaniga,883364555.0,882837596.0,None,2019-02-14 23:43:55.498175,https://en.wikipedia.org/w/index.php?diff=8833...,Car rush 1
7946,edit,30.0,N,NaN,[],False,False,False,True,False,[],Main,Inun-onan,NaN,883364557.0,[[WP:AES|←]]Redirected page to [[Paksiw#Inun-u...,2019-02-14 23:43:55.966439,https://en.wikipedia.org/w/index.php?oldid=883...,Obsidian Soul
7947,edit,-1.0,None,"{'city': 'Miami', 'country_name': 'United Stat...",[],True,False,False,False,False,[],Main,W. A. Martin,883364556.0,803817668.0,None,2019-02-14 23:43:56.318894,https://en.wikipedia.org/w/index.php?diff=8833...,2601:586:4104:6907:1E1:4F96:64E6:D917
7948,edit,7.0,MB,NaN,[],False,True,True,False,False,[],Main,Bruce Mayrock,883364558.0,878429616.0,Bot: Fixing double redirect to [[Nigerian Civi...,2019-02-14 23:43:57.136671,https://en.wikipedia.org/w/index.php?diff=8833...,EmausBot


The dataset was collected over an hour long period from approximately 22:43-23:43 14th February 2019.

In [6]:
print("Starting timestamp", data.timestamp.min())
print("Ending timestamp", data.timestamp.max())

Starting timestamp 2019-02-14 22:43:57.810307
Ending timestamp 2019-02-14 23:43:57.136671


In [7]:
# TODO: show edits to English language wikipedia articles only

Geolocation information is only available for anonymous or unregistered users, as the following three cells demonstrate.

In [8]:
data.is_anon.value_counts()

False    6636
True     1313
Name: is_anon, dtype: int64

In [9]:
data.geo_ip.isnull().value_counts()

True     6636
False    1313
Name: geo_ip, dtype: int64

In [10]:
data[data.is_anon == False].geo_ip.isnull().value_counts()

True    6636
Name: geo_ip, dtype: int64

In [11]:
# should be equivalent to is_anon flag - yes!
# also have several anonymous user records without geolocation information
registered_users = data[data.geo_ip.isnull()]
unregistered_users = data[(~data.geo_ip.isnull()) & (data.geo_ip != {})]
unregistered_users.reset_index(drop=True, inplace=True)

In [12]:
unregistered_users.shape

(1305, 19)

In [13]:
unregistered_users.head()

,action,change_size,flags,geo_ip,hashtags,is_anon,is_bot,is_minor,is_new,is_unpatrolled,mentions,ns,page_title,parent_rev_id,rev_id,summary,timestamp,url,user
0,hit,NaN,None,"{'city': 'San Jose', 'country_name': 'United S...",[],True,False,False,False,False,[],Special,Special:Log/abusefilter,NaN,NaN,143.254.206.247 triggered [[Special:AbuseFilte...,2019-02-14 22:44:02.724088,NaN,143.254.206.247
1,hit,NaN,None,"{'city': 'San Jose', 'country_name': 'United S...",[],True,False,False,False,False,[],Special,Special:Log/abusefilter,NaN,NaN,143.254.206.247 triggered [[Special:AbuseFilte...,2019-02-14 22:44:02.725470,NaN,143.254.206.247
2,edit,26.0,None,"{'city': 'Makati City', 'country_name': 'Phili...",[],True,False,False,False,False,[],Main,DuJour Media,883357340.0,880394349.0,None,2019-02-14 22:44:04.006324,https://en.wikipedia.org/w/index.php?diff=8833...,180.190.58.14
3,edit,26.0,None,"{'city': None, 'country_name': 'Philippines', ...",[],True,False,False,False,False,[],Main,St. Dominic College of Asia,883357344.0,883321675.0,None,2019-02-14 22:44:06.410434,https://en.wikipedia.org/w/index.php?diff=8833...,49.144.7.4
4,edit,1558.0,None,"{'city': 'Williamstown', 'country_name': 'Unit...",[],True,False,False,False,False,[],Main,3Below: Tales of Arcadia,883357347.0,882638905.0,/* Episodes */,2019-02-14 22:44:07.639533,https://en.wikipedia.org/w/index.php?diff=8833...,172.79.38.250


In [14]:
unregistered_users.action.value_counts()

edit    1045
hit      260
Name: action, dtype: int64

In [15]:
unregistered_users.ns.value_counts()

Main         999
Special      260
Talk          18
Template       7
Project        6
Draft          5
User talk      5
Category       2
User           1
Help           1
Portal         1
Name: ns, dtype: int64

In [16]:
unregistered_users.head()

,action,change_size,flags,geo_ip,hashtags,is_anon,is_bot,is_minor,is_new,is_unpatrolled,mentions,ns,page_title,parent_rev_id,rev_id,summary,timestamp,url,user
0,hit,NaN,None,"{'city': 'San Jose', 'country_name': 'United S...",[],True,False,False,False,False,[],Special,Special:Log/abusefilter,NaN,NaN,143.254.206.247 triggered [[Special:AbuseFilte...,2019-02-14 22:44:02.724088,NaN,143.254.206.247
1,hit,NaN,None,"{'city': 'San Jose', 'country_name': 'United S...",[],True,False,False,False,False,[],Special,Special:Log/abusefilter,NaN,NaN,143.254.206.247 triggered [[Special:AbuseFilte...,2019-02-14 22:44:02.725470,NaN,143.254.206.247
2,edit,26.0,None,"{'city': 'Makati City', 'country_name': 'Phili...",[],True,False,False,False,False,[],Main,DuJour Media,883357340.0,880394349.0,None,2019-02-14 22:44:04.006324,https://en.wikipedia.org/w/index.php?diff=8833...,180.190.58.14
3,edit,26.0,None,"{'city': None, 'country_name': 'Philippines', ...",[],True,False,False,False,False,[],Main,St. Dominic College of Asia,883357344.0,883321675.0,None,2019-02-14 22:44:06.410434,https://en.wikipedia.org/w/index.php?diff=8833...,49.144.7.4
4,edit,1558.0,None,"{'city': 'Williamstown', 'country_name': 'Unit...",[],True,False,False,False,False,[],Main,3Below: Tales of Arcadia,883357347.0,882638905.0,/* Episodes */,2019-02-14 22:44:07.639533,https://en.wikipedia.org/w/index.php?diff=8833...,172.79.38.250


In [17]:
geo_ip = json_normalize(unregistered_users["geo_ip"].tolist())
unregistered_users = unregistered_users.join(geo_ip, how="inner")

In [18]:
unregistered_users.drop(["geo_ip"], axis=1, inplace=True)

In [19]:
unregistered_users.head()

,action,change_size,flags,hashtags,is_anon,is_bot,is_minor,is_new,is_unpatrolled,mentions,...,rev_id,summary,timestamp,url,user,city,country_name,latitude,longitude,region_name
0,hit,NaN,None,[],True,False,False,False,False,[],...,NaN,143.254.206.247 triggered [[Special:AbuseFilte...,2019-02-14 22:44:02.724088,NaN,143.254.206.247,San Jose,United States,37.2972,-121.8868,California
1,hit,NaN,None,[],True,False,False,False,False,[],...,NaN,143.254.206.247 triggered [[Special:AbuseFilte...,2019-02-14 22:44:02.725470,NaN,143.254.206.247,San Jose,United States,37.2972,-121.8868,California
2,edit,26.0,None,[],True,False,False,False,False,[],...,880394349.0,None,2019-02-14 22:44:04.006324,https://en.wikipedia.org/w/index.php?diff=8833...,180.190.58.14,Makati City,Philippines,14.5501,121.0360,National Capital Region
3,edit,26.0,None,[],True,False,False,False,False,[],...,883321675.0,None,2019-02-14 22:44:06.410434,https://en.wikipedia.org/w/index.php?diff=8833...,49.144.7.4,None,Philippines,14.5955,120.9721,None
4,edit,1558.0,None,[],True,False,False,False,False,[],...,882638905.0,/* Episodes */,2019-02-14 22:44:07.639533,https://en.wikipedia.org/w/index.php?diff=8833...,172.79.38.250,Williamstown,United States,43.4275,-75.9090,New York


In [20]:
unregistered_edits = unregistered_users[unregistered_users.action == 'edit']
unregistered_edits.shape

(1045, 23)

In [21]:
unregistered_edits.country_name.value_counts()

United States             567
United Kingdom            135
Canada                     68
Ukraine                    27
Australia                  22
Spain                      17
Indonesia                  16
Germany                    16
Philippines                13
Brazil                     13
New Zealand                12
Chile                      10
Hong Kong                  10
Italy                      10
Greece                     10
India                       8
Serbia                      8
Ireland                     7
Netherlands                 5
Romania                     4
France                      4
Sweden                      4
Mexico                      3
Bosnia and Herzegovina      3
Poland                      3
Belgium                     3
Denmark                     3
Macedonia                   2
Dominican Republic          2
Republic of Korea           2
Russia                      2
South Africa                2
Venezuela                   2
Albania   

In [22]:
unregistered_edits.city.value_counts()

Vinnytsya                      25
London                         24
Cherry Hill                    22
New York                       16
Kaneohe                        15
Los Angeles                    12
Brooklyn                       11
Granada                        10
Central District               10
Birmingham                     10
Belgrade                        8
Ottawa                          7
Granite Bay                     6
Eugene                          6
Vitulazio                       5
South Portland                  5
Don Bosco Executive Village     5
Woodstock                       5
Porto Alegre                    5
Letha                           5
Wurruk                          5
Mechanicsburg                   5
Long Branch                     5
Chapel Hill                     5
Buffalo                         5
Gainesville                     4
Aurora                          4
Seattle                         4
Olathe                          4
Southampton   

In [23]:
unregistered_edits.region_name.value_counts()

England                       72
California                    50
New York                      45
New Jersey                    39
Florida                       26
Vinnyts'ka Oblast'            25
Pennsylvania                  25
Hawaii                        15
Illinois                      14
Massachusetts                 13
Victoria                      12
Wisconsin                     11
Tennessee                     11
Texas                         11
Andalusia                     10
Virginia                      10
Maryland                      10
Ontario                        9
Colorado                       9
North Carolina                 8
Washington                     8
East Java                      7
Oregon                         7
Region de Valparaiso           7
Calabarzon                     6
Georgia                        6
Campania                       5
Ohio                           5
Rio Grande do Sul              5
Maine                          5
          

In [24]:
# unregistered_edits[["country_name", "timestamp"]]

In [25]:
unregistered_edits.page_title

2                                            DuJour Media
3                             St. Dominic College of Asia
4                                3Below: Tales of Arcadia
5                                          V for Vendetta
6                                         Fela In Versace
7                                    Flag of Saudi Arabia
8                 Police Athletic League of New York City
9                                               Manta ray
10                                     Oswald (TV series)
11                                           Margie Hines
12                                          Michael Kogan
13                                            J.C. Luning
14                                     Marine chronometer
15                                  Talk:List of films: C
16                                  Bram Stoker's Dracula
17                                           DuJour Media
18                     Lieutenants Protection Association
19            

In [26]:
unregistered_edits[~unregistered_edits.summary.isnull()].summary

4                                          /* Episodes */
6                                      Removed propaganda
7       Change the word “God” to “Allah” because that ...
8                            /* Relationship with NYPD */
10                                         /* Episodes */
12                                           /* Career */
15                       /* The Contractor */ new section
16                                             /* Plot */
19      /* Semi-protected edit request on 14 February ...
21                                          /* Results */
22                                        /* Biography */
23                                         /* Novellas */
25      He had two brothers, Frank Arthur (b 1908) and...
26      /* Date */ Removed the word "Anyways" for bein...
27                                          /* Results */
31                    /* 3. Quia respexit */Fixed mistake
32                                      /* Filmography */
34            

In [27]:
unregistered_edits[unregistered_edits.url.apply(lambda x: "https://en.wikipedia.org" in x)].shape

(1045, 23)

In [28]:
unregistered_edits[["page_title", "summary", "url", "country_name"]].sort_values("country_name")

,page_title,summary,url,country_name
878,Graham Gardner,/* Inventing Elliot */Fixed typo,https://en.wikipedia.org/w/index.php?diff=8833...,Albania
865,Graham Gardner,/* Inventing Elliot */Added content,https://en.wikipedia.org/w/index.php?diff=8833...,Albania
948,You Belong with Me,None,https://en.wikipedia.org/w/index.php?diff=8833...,Argentina
599,ANSES,None,https://en.wikipedia.org/w/index.php?diff=8833...,Argentina
894,Asami Sanada,"Garuru doesn't have a family name and ""Bokerdo...",https://en.wikipedia.org/w/index.php?diff=8833...,Australia
596,Selimiye Mosque,/* Description */,https://en.wikipedia.org/w/index.php?diff=8833...,Australia
1279,Hughes (surname),w[vn89b 04hOZIUF0AWHGAEGAEFJFWG]J]AEJGEOJG]wjg...,https://en.wikipedia.org/w/index.php?diff=8833...,Australia
709,Trout,/* Species */,https://en.wikipedia.org/w/index.php?diff=8833...,Australia
424,Joseph Charles Bequaert,/* Career */Fixed typo,https://en.wikipedia.org/w/index.php?diff=8833...,Australia
464,"Raukkan, South Australia",None,https://en.wikipedia.org/w/index.php?diff=8833...,Australia


In [36]:
unregistered_edits[unregistered_edits["page_title"] == "Emma Mackey"].iloc[0].url

'https://en.wikipedia.org/w/index.php?diff=883360600&oldid=883353413'

In [29]:
unregistered_edits[["page_title", "summary", "country_name"]]

,page_title,summary,country_name
2,DuJour Media,None,Philippines
3,St. Dominic College of Asia,None,Philippines
4,3Below: Tales of Arcadia,/* Episodes */,United States
5,V for Vendetta,None,United States
6,Fela In Versace,Removed propaganda,South Africa
7,Flag of Saudi Arabia,Change the word “God” to “Allah” because that ...,United States
8,Police Athletic League of New York City,/* Relationship with NYPD */,United States
9,Manta ray,None,United States
10,Oswald (TV series),/* Episodes */,United States
11,Margie Hines,None,Serbia


In [30]:
x = np.array([datetime.datetime(2013, 9, 28, i, 0) for i in range(24)])
y = np.random.randint(100, size=x.shape)

plt.plot(,y)
plt.show()

SyntaxError: invalid syntax (<ipython-input-30-0669855f8b54>, line 4)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [ ]:
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution=None,
            width=8E6, height=8E6, 
            lat_0=45, lon_0=-100,)
m.etopo(scale=0.5, alpha=0.5)

# Map (long, lat) to (x, y) for plotting
x, y = m(-122.3, 47.6)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, ' Seattle', fontsize=12);

In [ ]:
fig = plt.figure(figsize=(8, 6), edgecolor='w')
m = Basemap(projection='cyl', resolution=None,
            llcrnrlat=-90, urcrnrlat=90,
            llcrnrlon=-180, urcrnrlon=180, )
draw_map(m)

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
# set up orthographic map projection with
# perspective of satellite looking down at 50N, 100W.
# use low resolution coastlines.
map = Basemap(projection='ortho',lat_0=45,lon_0=-100,resolution='l')
# draw coastlines, country boundaries, fill continents.
map.drawcoastlines(linewidth=0.25)
map.drawcountries(linewidth=0.25)
map.fillcontinents(color='coral',lake_color='aqua')
# draw the edge of the map projection region (the projection limb)
map.drawmapboundary(fill_color='aqua')
# draw lat/lon grid lines every 30 degrees.
map.drawmeridians(np.arange(0,360,30))
map.drawparallels(np.arange(-90,90,30))
# make up some data on a regular lat/lon grid.
nlats = 73; nlons = 145; delta = 2.*np.pi/(nlons-1)
lats = (0.5*np.pi-delta*np.indices((nlats,nlons))[0,:,:])
lons = (delta*np.indices((nlats,nlons))[1,:,:])
wave = 0.75*(np.sin(2.*lats)**8*np.cos(4.*lons))
mean = 0.5*np.cos(2.*lats)*((np.sin(2.*lats))**2 + 2.)
# compute native map projection coordinates of lat/lon grid.
x, y = map(lons*180./np.pi, lats*180./np.pi)
# contour data over the map.
cs = map.contour(x,y,wave+mean,15,linewidths=1.5)
plt.title('contour lines over filled continent background')
plt.show()